-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Delta Lake Versioning, Optimization, and Vacuuming

This notebook provides a hands-on review of some of the more esoteric features Delta Lake brings to the data lakehouse.

## Learning Objectives
By the end of this lab, you should be able to:
- Review table history
- Query previous table versions and rollback a table to a specific version
- Perform file compaction and Z-order indexing
- Preview files marked for permanent deletion and commit these deletes

## Setup
Run the following script to setup necessary variables and clear out past runs of this notebook. Note that re-executing this cell will allow you to start the lab over.

In [0]:
%run ../Includes/Classroom-Setup-2.4L

Python interpreter will be restarted.
Python interpreter will be restarted.

Dropping the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l"
Removing the working directory "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l"

Creating the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l"

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l
 DA.paths.user_db: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db

Predefined tables in dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l:
 -none-

Setup completed in 2 seconds

## Recreate the History of your Bean Collection

This lab picks up where the last lab left off. The cell below condenses all the operations from the last lab into a single cell (other than the final **`DROP TABLE`** statement).

For quick reference, the schema of the **`beans`** table created is:

| Field Name | Field type |
| --- | --- |
| name | STRING |
| color | STRING |
| grams | FLOAT |
| delicious | BOOLEAN |

In [0]:
%sql
CREATE TABLE beans 
(name STRING, color STRING, grams FLOAT, delicious BOOLEAN);

INSERT INTO beans VALUES
("black", "black", 500, true),
("lentils", "brown", 1000, true),
("jelly", "rainbow", 42.5, false);

INSERT INTO beans VALUES
('pinto', 'brown', 1.5, true),
('green', 'green', 178.3, true),
('beanbag chair', 'white', 40000, false);

UPDATE beans
SET delicious = true
WHERE name = "jelly";

UPDATE beans
SET grams = 1500
WHERE name = 'pinto';

DELETE FROM beans
WHERE delicious = false;

CREATE OR REPLACE TEMP VIEW new_beans(name, color, grams, delicious) AS VALUES
('black', 'black', 60.5, true),
('lentils', 'green', 500, true),
('kidney', 'red', 387.2, true),
('castor', 'brown', 25, false);

MERGE INTO beans a
USING new_beans b
ON a.name=b.name AND a.color = b.color
WHEN MATCHED THEN
  UPDATE SET grams = a.grams + b.grams
WHEN NOT MATCHED AND b.delicious = true THEN
  INSERT *;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,1,0,2


## Review the Table History

Delta Lake's transaction log stores information about each transaction that modifies a table's contents or settings.

Review the history of the **`beans`** table below.

In [0]:
%sql
-- TODO
describe history beans

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
6,2022-07-28T07:16:39.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,MERGE,"Map(predicate -> ((a.name = b.name) AND (a.color = b.color)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""predicate"":""b.delicious"",""actionType"":""insert""}])",null,List(2331746562402643),0725-045645-b5m629fz,5,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 3, executionTimeMs -> 1616, numTargetRowsInserted -> 2, scanTimeMs -> 965, numTargetRowsUpdated -> 1, numOutputRows -> 3, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 1, rewriteTimeMs -> 529)",null,Databricks-Runtime/10.4.x-scala2.12
5,2022-07-28T07:16:36.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,DELETE,"Map(predicate -> [""(NOT spark_catalog.dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l.beans.delicious)""])",null,List(2331746562402643),0725-045645-b5m629fz,4,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 551, numDeletedRows -> 1, scanTimeMs -> 398, numAddedFiles -> 0, rewriteTimeMs -> 153)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-07-28T07:16:34.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29807 = pinto)),null,List(2331746562402643),0725-045645-b5m629fz,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 492, scanTimeMs -> 71, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 421)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-07-28T07:16:32.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29283 = jelly)),null,List(2331746562402643),0725-045645-b5m629fz,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 614, scanTimeMs -> 112, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 502)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-07-28T07:16:30.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2331746562402643),0725-045645-b5m629fz,1,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3908)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-07-28T07:16:29.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2331746562402643),0725-045645-b5m629fz,0,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3880)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-07-28T07:16:25.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2331746562402643),0725-045645-b5m629fz,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


If all the previous operations were completed as described you should see 7 versions of the table (**NOTE**: Delta Lake versioning starts with 0, so the max version number will be 6).

The operations should be as follows:

| version | operation |
| --- | --- |
| 0 | CREATE TABLE |
| 1 | WRITE |
| 2 | WRITE |
| 3 | UPDATE |
| 4 | UPDATE |
| 5 | DELETE |
| 6 | MERGE |

The **`operationsParameters`** column will let you review predicates used for updates, deletes, and merges. The **`operationMetrics`** column indicates how many rows and files are added in each operation.

Spend some time reviewing the Delta Lake history to understand which table version matches with a given transaction.

**NOTE**: The **`version`** column designates the state of a table once a given transaction completes. The **`readVersion`** column indicates the version of the table an operation executed against. In this simple demo (with no concurrent transactions), this relationship should always increment by 1.

## Query a Specific Version

After reviewing the table history, you decide you want to view the state of your table after your very first data was inserted.

Run the query below to see this.

In [0]:
%sql
SELECT * FROM beans VERSION AS OF 6

name,color,grams,delicious
lentils,brown,1000.0,true
jelly,rainbow,42.5,true
lentils,green,500.0,true
green,green,178.3,true
pinto,brown,1500.0,true
black,black,560.5,true
kidney,red,387.2,true


And now review the current state of your data.

In [0]:
%sql
SELECT * FROM beans

name,color,grams,delicious
lentils,brown,1000.0,true
jelly,rainbow,42.5,true
lentils,green,500.0,true
green,green,178.3,true
pinto,brown,1500.0,true
black,black,560.5,true
kidney,red,387.2,true


### You want to review the weights of your beans before you deleted any records.

Fill in the statement below to register a temporary view of the version just before data was deleted, then run the following cell to query the view.

In [0]:
%sql
-- TODO
CREATE OR REPLACE TEMP VIEW pre_delete_vw AS
  select * from beans version as of 4

In [0]:
%sql
SELECT * FROM pre_delete_vw

name,color,grams,delicious
beanbag chair,white,40000.0,false
lentils,brown,1000.0,true
jelly,rainbow,42.5,true
black,black,500.0,true
green,green,178.3,true
pinto,brown,1500.0,true


Run the cell below to check that you have captured the correct version.

In [0]:
%python
assert spark.table("pre_delete_vw"), "Make sure you have registered the temporary view with the provided name `pre_delete_vw`"
assert spark.table("pre_delete_vw").count() == 6, "Make sure you're querying a version of the table with 6 records"
assert spark.table("pre_delete_vw").selectExpr("int(sum(grams))").first()[0] == 43220, "Make sure you query the version of the table after updates were applied"

## Restore a Previous Version

Apparently there was a misunderstanding; the beans your friend gave you that you merged into your collection were not intended for you to keep.

Revert your table to the version before this **`MERGE`** statement completed.

In [0]:
%sql
-- TODO
restore table beans to version as of 5

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
6448,5,3,1,3857,1284


In [0]:
%sql
describe history beans

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2022-07-28T07:16:47.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,RESTORE,"Map(version -> 5, timestamp -> null)",null,List(2331746562402643),0725-045645-b5m629fz,6,Serializable,false,"Map(numRestoredFiles -> 1, removedFilesSize -> 3857, numRemovedFiles -> 3, restoredFilesSize -> 1284, numOfFilesAfterRestore -> 5, tableSizeAfterRestore -> 6448)",null,Databricks-Runtime/10.4.x-scala2.12
6,2022-07-28T07:16:39.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,MERGE,"Map(predicate -> ((a.name = b.name) AND (a.color = b.color)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""predicate"":""b.delicious"",""actionType"":""insert""}])",null,List(2331746562402643),0725-045645-b5m629fz,5,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 3, executionTimeMs -> 1616, numTargetRowsInserted -> 2, scanTimeMs -> 965, numTargetRowsUpdated -> 1, numOutputRows -> 3, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 1, rewriteTimeMs -> 529)",null,Databricks-Runtime/10.4.x-scala2.12
5,2022-07-28T07:16:36.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,DELETE,"Map(predicate -> [""(NOT spark_catalog.dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l.beans.delicious)""])",null,List(2331746562402643),0725-045645-b5m629fz,4,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 551, numDeletedRows -> 1, scanTimeMs -> 398, numAddedFiles -> 0, rewriteTimeMs -> 153)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-07-28T07:16:34.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29807 = pinto)),null,List(2331746562402643),0725-045645-b5m629fz,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 492, scanTimeMs -> 71, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 421)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-07-28T07:16:32.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29283 = jelly)),null,List(2331746562402643),0725-045645-b5m629fz,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 614, scanTimeMs -> 112, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 502)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-07-28T07:16:30.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2331746562402643),0725-045645-b5m629fz,1,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3908)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-07-28T07:16:29.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2331746562402643),0725-045645-b5m629fz,0,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3880)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-07-28T07:16:25.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2331746562402643),0725-045645-b5m629fz,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


Review the history of your table. Make note of the fact that restoring to a previous version adds another table version.

In [0]:
%sql
DESCRIBE HISTORY beans

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2022-07-28T07:16:47.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,RESTORE,"Map(version -> 5, timestamp -> null)",null,List(2331746562402643),0725-045645-b5m629fz,6,Serializable,false,"Map(numRestoredFiles -> 1, removedFilesSize -> 3857, numRemovedFiles -> 3, restoredFilesSize -> 1284, numOfFilesAfterRestore -> 5, tableSizeAfterRestore -> 6448)",null,Databricks-Runtime/10.4.x-scala2.12
6,2022-07-28T07:16:39.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,MERGE,"Map(predicate -> ((a.name = b.name) AND (a.color = b.color)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""predicate"":""b.delicious"",""actionType"":""insert""}])",null,List(2331746562402643),0725-045645-b5m629fz,5,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 3, executionTimeMs -> 1616, numTargetRowsInserted -> 2, scanTimeMs -> 965, numTargetRowsUpdated -> 1, numOutputRows -> 3, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 1, rewriteTimeMs -> 529)",null,Databricks-Runtime/10.4.x-scala2.12
5,2022-07-28T07:16:36.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,DELETE,"Map(predicate -> [""(NOT spark_catalog.dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l.beans.delicious)""])",null,List(2331746562402643),0725-045645-b5m629fz,4,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 551, numDeletedRows -> 1, scanTimeMs -> 398, numAddedFiles -> 0, rewriteTimeMs -> 153)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-07-28T07:16:34.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29807 = pinto)),null,List(2331746562402643),0725-045645-b5m629fz,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 492, scanTimeMs -> 71, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 421)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-07-28T07:16:32.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29283 = jelly)),null,List(2331746562402643),0725-045645-b5m629fz,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 614, scanTimeMs -> 112, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 502)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-07-28T07:16:30.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2331746562402643),0725-045645-b5m629fz,1,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3908)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-07-28T07:16:29.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2331746562402643),0725-045645-b5m629fz,0,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3880)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-07-28T07:16:25.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2331746562402643),0725-045645-b5m629fz,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


In [0]:
%python
last_tx = spark.conf.get("spark.databricks.delta.lastCommitVersionInSession")
assert spark.sql(f"DESCRIBE HISTORY beans").select("operation").first()[0] == "RESTORE", "Make sure you reverted your table with the `RESTORE` keyword"
assert spark.table("beans").count() == 5, "Make sure you reverted to the version after deleting records but before merging"

# File Compaction
Looking at the transaction metrics during your reversion, you are surprised you have some many files for such a small collection of data.

While indexing on a table of this size is unlikely to improve performance, you decide to add a Z-order index on the **`name`** field in anticipation of your bean collection growing exponentially over time.

Use the cell below to perform file compaction and Z-order indexing.

In [0]:
%sql
-- TODO
optimize beans
zorder by name

path,metrics
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans,"List(1, 5, List(1381, 1381, 1381.0, 1, 1381), List(1284, 1298, 1289.6, 5, 6448), 0, List(minCubeSize(107374182400), List(0, 0), List(5, 6448), 0, List(5, 6448), 1, null), 1, 5, 0, false)"


Your data should have been compacted to a single file; confirm this manually by running the following cell.

In [0]:
%sql
DESCRIBE DETAIL beans

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,a02e2ab0-817b-4c81-aaa2-087aa510035b,dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l.beans,null,dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans,2022-07-28T07:16:25.214+0000,2022-07-28T07:19:46.000+0000,List(),1,1381,Map(),1,2


Run the cell below to check that you've successfully optimized and indexed your table.

In [0]:
%python
last_tx = spark.sql("DESCRIBE HISTORY beans").first()
assert last_tx["operation"] == "OPTIMIZE", "Make sure you used the `OPTIMIZE` command to perform file compaction"
assert last_tx["operationParameters"]["zOrderBy"] == '["name"]', "Use `ZORDER BY name` with your optimize command to index your table"

# Cleaning Up Stale Data Files

**You know that while all your data now resides in 1 data file**, the data files from previous versions of your table are still being stored alongside this. You wish to remove these files and remove access to previous versions of the table by running **`VACUUM`** on the table.

Executing **`VACUUM`** performs garbage cleanup on the table directory. By default, a retention threshold of 7 days will be enforced.

The cell below modifies some Spark configurations. The first command overrides the retention threshold check to allow us to demonstrate permanent removal of data. 

**NOTE**: Vacuuming a production table with a short retention can lead to data corruption and/or failure of long-running queries. This is for demonstration purposes only and extreme caution should be used when disabling this setting.

The second command sets **`spark.databricks.delta.vacuum.logging.enabled`** to **`true`** to ensure that the **`VACUUM`** operation is recorded in the transaction log.

**NOTE**: Because of slight differences in storage protocols on various clouds, logging **`VACUUM`** commands is not on by default for some clouds as of DBR 9.1.

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
SET spark.databricks.delta.vacuum.logging.enabled = true;

key,value
spark.databricks.delta.vacuum.logging.enabled,true


Before permanently deleting data files, review them manually using the **`DRY RUN`** option.

In [0]:
%sql
VACUUM beans RETAIN 0 HOURS DRY RUN

path
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00000-1c63cb88-c807-4a87-878e-4dc58eaff239-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00000-3afd8088-6b01-4a9b-9deb-83f761c98b55-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00000-7ab7bd6b-554b-43a5-9e44-05ae60589d34-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00000-8a82ba00-5cc2-4c61-ba8c-95d599bc8a4e-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00000-a595fddb-dc24-4064-852a-5973c68f9bb3-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00001-184f84de-212a-4164-9b67-0820ef2e2856-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00001-2e266b74-e8ec-4ca6-a384-e6843ad88ab7-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00001-738ed63e-dd72-4ab4-b6d7-00e03fa0c7d8-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00002-0e61cb65-fcd5-47cb-8160-03411f903b25-c000.snappy.parquet
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans/part-00002-1be1a8c7-205c-44ae-9176-8795e746740e-c000.snappy.parquet


All data files not in the current version of the table will be shown in the preview above.

Run the command again without **`DRY RUN`** to permanently delete these files.

**NOTE**: All previous versions of the table will no longer be accessible.

In [0]:
%sql
VACUUM beans RETAIN 0 HOURS

path
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l/2_4l.db/beans


Because **`VACUUM`** can be such a destructive act for important datasets, it's always a good idea to turn the retention duration check back on. Run the cell below to reactive this setting.

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = true

key,value
spark.databricks.delta.retentionDurationCheck.enabled,true


Note that the table history will indicate the user that completed the **`VACUUM`** operation, the number of files deleted, and log that the retention check was disabled during this operation.

In [0]:
%sql
DESCRIBE HISTORY beans

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
10,2022-07-28T07:26:55.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,VACUUM END,Map(status -> COMPLETED),null,List(2331746562402643),0725-045645-b5m629fz,9,SnapshotIsolation,true,"Map(numDeletedFiles -> 11, numVacuumedDirectories -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
9,2022-07-28T07:26:52.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,VACUUM START,"Map(retentionCheckEnabled -> false, specifiedRetentionMillis -> 0, defaultRetentionMillis -> 604800000)",null,List(2331746562402643),0725-045645-b5m629fz,8,SnapshotIsolation,true,Map(numFilesToDelete -> 11),null,Databricks-Runtime/10.4.x-scala2.12
8,2022-07-28T07:19:46.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""name""], batchId -> 0, auto -> false)",null,List(2331746562402643),0725-045645-b5m629fz,7,SnapshotIsolation,false,"Map(numRemovedFiles -> 5, numRemovedBytes -> 6448, p25FileSize -> 1381, minFileSize -> 1381, numAddedFiles -> 1, maxFileSize -> 1381, p75FileSize -> 1381, p50FileSize -> 1381, numAddedBytes -> 1381)",null,Databricks-Runtime/10.4.x-scala2.12
7,2022-07-28T07:16:47.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,RESTORE,"Map(version -> 5, timestamp -> null)",null,List(2331746562402643),0725-045645-b5m629fz,6,Serializable,false,"Map(numRestoredFiles -> 1, removedFilesSize -> 3857, numRemovedFiles -> 3, restoredFilesSize -> 1284, numOfFilesAfterRestore -> 5, tableSizeAfterRestore -> 6448)",null,Databricks-Runtime/10.4.x-scala2.12
6,2022-07-28T07:16:39.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,MERGE,"Map(predicate -> ((a.name = b.name) AND (a.color = b.color)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""predicate"":""b.delicious"",""actionType"":""insert""}])",null,List(2331746562402643),0725-045645-b5m629fz,5,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 3, executionTimeMs -> 1616, numTargetRowsInserted -> 2, scanTimeMs -> 965, numTargetRowsUpdated -> 1, numOutputRows -> 3, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 1, rewriteTimeMs -> 529)",null,Databricks-Runtime/10.4.x-scala2.12
5,2022-07-28T07:16:36.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,DELETE,"Map(predicate -> [""(NOT spark_catalog.dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l.beans.delicious)""])",null,List(2331746562402643),0725-045645-b5m629fz,4,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 551, numDeletedRows -> 1, scanTimeMs -> 398, numAddedFiles -> 0, rewriteTimeMs -> 153)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-07-28T07:16:34.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29807 = pinto)),null,List(2331746562402643),0725-045645-b5m629fz,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 492, scanTimeMs -> 71, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 421)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-07-28T07:16:32.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,UPDATE,Map(predicate -> (name#29283 = jelly)),null,List(2331746562402643),0725-045645-b5m629fz,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 614, scanTimeMs -> 112, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 502)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-07-28T07:16:30.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2331746562402643),0725-045645-b5m629fz,1,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes 

Query your table again to confirm you still have access to the current version.

In [0]:
%sql
SELECT * FROM beans

name,color,grams,delicious
jelly,rainbow,42.5,true
lentils,brown,1000.0,true
pinto,brown,1500.0,true
black,black,500.0,true
green,green,178.3,true


<img src="https://files.training.databricks.com/images/icon_warn_32.png"> Because Delta Cache stores copies of files queried in the current session on storage volumes deployed to your currently active cluster, you may still be able to temporarily access previous table versions (though systems should **not** be designed to expect this behavior). 

Restarting the cluster will ensure that these cached data files are permanently purged.

You can see an example of this by uncommenting and running the following cell that may, or may not, fail
(depending on the state of the cache).

In [0]:
%sql
--SELECT * FROM beans@v1

By completing this lab, you should now feel comfortable:
* Completing standard Delta Lake table creation and data manipulation commands
* Reviewing table metadata including table history
* Leverage Delta Lake versioning for snapshot queries and rollbacks
* Compacting small files and indexing tables
* Using **`VACUUM`** to review files marked for deletion and committing these deletes

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python
DA.cleanup()

Dropping the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_2_4l"
Removing the working directory "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/2.4l"

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>